In [ ]:
import sys,time
import import_ipynb
#from api.ParseWeb import ParseWeb

%run ../api/ParseWeb.ipynb

parser = ParseWeb( verbose=False, debug=False, printSites=False )

In [ ]:
%run ../api/classy990downloads.ipynb


year = 2017
uri='mongodb://mongo/irs990'

db = mongo990( uri=uri, year=year, dump=False)
        
db.verify()



In [ ]:
#for x in db.collection.find( 
#    { '$and': [{'/IRS990/WebsiteAddressTxt': { '$exists': True }}, 
#      {'/IRS990/WebsiteAddressTxt': { '$ne': 'n/a' }} ,
#      {'/IRS990/WebsiteAddressTxt': { '$ne': 'N/A' }} ]
#    },
#    { "_id": 1, "/IRS990/WebsiteAddressTxt": 1, 'OrganizationName':1 } ):

for x in db.collection.find( {},{ "_id": 1, "/IRS990/WebsiteAddressTxt": 1, 'OrganizationName':1 } ):
    name = x['OrganizationName']
    try:
        url = x['/IRS990/WebsiteAddressTxt']
    except:
        url = False
    
    
    padding = '                                              '
    sys.stdout.write( '%s%s%s\r' % (padding,padding,padding) )
    sys.stdout.flush() 
    
    
    sys.stdout.write( '%s => %s ' % ( name, url ) )

    if not url or 'N/A' in url or 'n/a' in url:
        sys.stdout.write( '*** MISSING URL *** ')
        
    else:
        websites, text = parser.text( url )
        sys.stdout.write( text )

        if text:
            try:
                db.collection.update_one( {"_id" : x['_id']}, {"$set":{'/IRS990/WebsiteAddressTxt':websites, 'WebTxt':text}}, upsert=True )
            except Exception as e:
                sys.stdout.write( 'ERROR: Problem during MongoDB insert. %s' % e )

        else:
            sys.stdout.write( ' *** NO DATA ***' )

    sys.stdout.write( '\r' )
    sys.stdout.flush() 